# Loading Necessary Libraries and Models

In [2]:
# !pip install PyPDF2
# !pip install nltk
# !pip install wordcloud
# !pip install PyMuPDF
# from spacy.cli import download
# download("en_core_web_sm")
# !pip install pycryptodome
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# !pip install keybert
# !pip install -U sentence-transformers
# !pip install transformers

In [3]:
import PyPDF2
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import spacy
import re
import matplotlib.pyplot as plt
from collections import Counter
import fitz  # PyMuPDF
from wordcloud import WordCloud
from spacy import displacy
from nltk.tokenize import word_tokenize
import os
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
stop = set(stopwords.words("english"))
from nltk import sent_tokenize, word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.util import ngrams
nlp = spacy.load("en_core_web_sm")
import ast
from sentence_transformers import SentenceTransformer, util
Sbert = SentenceTransformer('all-MiniLM-L6-v2')
import io
from PIL import Image
import requests
from transformers import pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Extracting PMBOK

In [33]:
pdf_file_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/PMBOK6-2017.pdf"
pdf_document = fitz.open(pdf_file_path)
extracted_text = ''
for page_num in range(430,493): #395+35 -> 458+35
    page = pdf_document[page_num]
    page_text = page.get_text()
    extracted_text += page_text
pdf_document.close()
# extracted_text

## Cleaning PMBOK

In [34]:
def clean_pmbok(extracted_text):
    patterns_and_replacements = [
        (r'Not For Distribution, Sale or Reproduction\.', ''),
        (r'Part 1 - Guide', ''),
        (r'\d{3}', ''),
        (r'(?<![a-zA-Z])u(?!.)', ''),
        (r'u ', ''),
        (r'\bn\b',''),
        (r'ﬁ', 'fi'),
        (r'\(', ''),
        (r'\)', ''),
        (r',', ''),
        (r'`', '')
        #(r'\n','')
    ]
    for pattern, replacement in patterns_and_replacements:
        extracted_text = re.sub(pattern, replacement, extracted_text)

    return extracted_text

## Section Extraction and Analysis

In [89]:
def extract_table_of_contents(pdf_path, start_page, end_page):
    pdf_document = fitz.open(pdf_path)
    table_of_contents = ""
    for page_num in range(start_page - 1, end_page):
        page = pdf_document.load_page(page_num)
        table_of_contents += page.get_text()
    pdf_document.close()
    return table_of_contents

table_of_contents_text = extract_table_of_contents(pdf_file_path, 11, 12)
print(table_of_contents_text[50:1000])


4 Develop Team................................................................................................................336
9.4.1 Develop Team: Inputs .......................................................................................339
9.4.2 Develop Team: Tools and Techniques ..............................................................340
9.4.3 Develop Team: Outputs ....................................................................................343
9.5 Manage Team................................................................................................................345
9.5.1 Manage Team: Inputs .......................................................................................347
9.5.2 Manage Team: Tools and Techniques ..............................................................348
9.5.3 Manage Team: Outputs ....................................................................................350
9.6 Contro


### Extracting RISK MANAGEMENT only

In [90]:
lines = table_of_contents_text.split('\n')
start = lines.index('11. PROJECT RISK MANAGEMENT .............................................................................................395')
finish = lines.index('12. PROJECT PROCUREMENT MANAGEMENT ............................................................................459')
lines = lines[start:finish]
print(lines)

['11. PROJECT RISK MANAGEMENT .............................................................................................395', '11.1 Plan Risk Management ..............................................................................................401', '11.1.1 Plan Risk Management: Inputs ......................................................................402', '11.1.2 Plan Risk Management: Tools and Techniques .............................................404', '11.1.3 Plan Risk Management: Outputs ...................................................................405', 'Not For Distribution, Sale or Reproduction.', 'VIII ', 'Table of Contents', '11.2 Identify Risks ..............................................................................................................409', '11.2.1 Identify Risks: Inputs .....................................................................................411', '11.2.2 Identify Risks: Tools and Techniques .....................................

### Cleaning Table of Contents

In [91]:
table_of_contents_text_clean = ''.join(lines)
table_of_contents_text_clean = table_of_contents_text_clean.replace('Not For Distribution, Sale or Reproduction','')
table_of_contents_text_clean = table_of_contents_text_clean.replace('Table of Contents','')

### Get titles

In [92]:
title_pattern = r'\s+([A-Za-z\s:]+)'
titles = []
title_matches = re.findall(title_pattern, table_of_contents_text_clean)
titles.extend(title_matches)
print(titles)

['PROJECT RISK MANAGEMENT ', 'Plan Risk Management ', 'Plan Risk Management: Inputs ', 'Plan Risk Management: Tools and Techniques ', 'Plan Risk Management: Outputs ', 'Identify Risks ', 'Identify Risks: Inputs ', 'Identify Risks: Tools and Techniques ', 'Identify Risks: Outputs ', 'Perform Qualitative Risk Analysis ', 'Perform Qualitative Risk Analysis: Inputs ', 'Perform Qualitative Risk Analysis: Tools and Techniques ', 'Perform Qualitative Risk Analysis: Outputs ', 'Perform Quantitative Risk Analysis ', 'Perform Quantitative Risk Analysis: Inputs ', 'Perform Quantitative Risk Analysis: Tools and Techniques ', 'Perform Quantitative Risk Analysis: Outputs ', 'Plan Risk Responses ', 'Plan Risk Responses: Inputs ', 'Plan Risk Responses: Tools and Techniques ', 'Plan Risk Responses: Outputs ', 'Implement Risk Responses ', 'Implement Risk Responses: Inputs ', 'Implement Risk Responses: Tools and Techniques ', 'Implement Risk Responses: Outputs', 'Monitor Risks ', 'Monitor Risks: Inputs '

### Get Page Numbers

In [93]:
page_number_pattern = r'\d{3}'
page_numbers = []
page_num_matches = re.findall(page_number_pattern, table_of_contents_text_clean)
page_numbers.extend(page_num_matches)
print(page_numbers)

['395', '401', '402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457']


### Get Sections

In [94]:
section_pattern = r'^(\d+(\.\d{1,3})*)'
sections = []
for line in lines:
    section_match = re.search(section_pattern, line)
    if section_match:
        sections.append(section_match.group(1))
print(sections)

['11', '11.1', '11.1.1', '11.1.2', '11.1.3', '11.2', '11.2.1', '11.2.2', '11.2.3', '11.3', '11.3.1', '11.3.2', '11.3.3', '11.4', '11.4.1', '11.4.2', '11.4.3', '11.5', '11.5.1', '11.5.2', '11.5.3', '11.6', '11.6.1', '11.6.2', '11.6.3', '11.7', '11.7.1', '11.7.2', '11.7.3']


### Verifying Lengths for each list

In [95]:
print(f' Sections : {len(sections)}')
print(f' Titles : {len(titles)}')
print(f' Page Numbers : {len(page_numbers)}')


 Sections : 29
 Titles : 29
 Page Numbers : 29


### Creating a Dataframe

In [108]:
data = {"Title": titles, "Starting_Page_Number": page_numbers, "Section": sections}
df = pd.DataFrame(data)
df.sample(12)

,Title,Starting_Page_Number,Section
13,Perform Quantitative Risk Analysis,430,11.4
23,Implement Risk Responses: Tools and Techniques,451,11.6.2
9,Perform Qualitative Risk Analysis,421,11.3
5,Identify Risks,411,11.2
17,Plan Risk Responses,439,11.5
10,Perform Qualitative Risk Analysis: Inputs,422,11.3.1
16,Perform Quantitative Risk Analysis: Outputs,437,11.4.3
24,Implement Risk Responses: Outputs,453,11.6.3
15,Perform Quantitative Risk Analysis: Tools and ...,436,11.4.2
21,Implement Risk Responses,450,11.6


In [ ]:
df.head()

,Title,Starting_Page_Number,Section
0,PROJECT RISK MANAGEMENT,395,11
1,Plan Risk Management,401,11.1
2,Plan Risk Management: Inputs,402,11.1.1
3,Plan Risk Management: Tools and Techniques,404,11.1.2
4,Plan Risk Management: Outputs,405,11.1.3


### Adding 'Ending Page Number' Column

In [109]:
page_numbers = df['Starting_Page_Number'].tolist()
end_pages = page_numbers
end_pages.pop(0)
end_pages.append('459')
print(end_pages)
df['Ending_Page_Number'] = end_pages

['402', '404', '405', '409', '411', '414', '417', '419', '421', '422', '427', '428', '430', '431', '436', '437', '439', '441', '447', '449', '450', '451', '451', '453', '455', '456', '457', '459', '459']


In [110]:
df['Title'] = df['Title'].str.rstrip()

In [111]:
df['Starting_Page_Number'] = df['Starting_Page_Number'].astype('int')
df['Ending_Page_Number'] = df['Ending_Page_Number'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Title                 29 non-null     object
 1   Starting_Page_Number  29 non-null     int64 
 2   Section               29 non-null     object
 3   Ending_Page_Number    29 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 1.0+ KB


### Content Extraction and Cleaning For Each Section

In [121]:
def content_extraction_each_section(pdf_file,Starting_Page_Number,Ending_Page_Number,decalage=0):
    pdf_document = fitz.open(pdf_file)
    extracted_text = ""
    for page_num in range(Starting_Page_Number +decalage, Ending_Page_Number+decalage):
        page = pdf_document.load_page(page_num)
        extracted_text += page.get_text()
    clean_text = clean_pmbok(extracted_text)
    pdf_document.close()
    return clean_text

In [122]:
def content_extraction_for_df(df):
    df['Content'] = df.apply(lambda x: content_extraction_each_section(pdf_file_path,x['Starting_Page_Number'],x['Ending_Page_Number'], 35),axis=1)
    return df

In [123]:
content_extraction_for_df(df)
df['Content'] = df['Content'].str.replace('\n','')
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content
0,PROJECT RISK MANAGEMENT,401,11,402,11.1 PLAN RISK MANAGEMENTPlan Risk Management ...
1,Plan Risk Management,402,11.1,404,• Project charter11.1 Plan RiskManagementEn...
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.2 PLAN RISK MANAGEMENT: TOOLS AND TECHNI...
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.3 PLAN RISK MANAGEMENT: OUTPUTS11.1.3.1 R...
4,Plan Risk Management Outputs,409,11.1.3,411,11.2 IDENTIFY RISKSIdentify Risks is the proce...


In [49]:
df.loc[2].Content

' • Project   charter11.1 Plan RiskManagementEnterprise/Organization4.1Develop ProjectCharter• Project charterProject documents• Stakeholder register• Enterprise environmental factors• Organizational process assetsProject ManagementPlanProject management plan• All componentsProjectManagementPlanProjectDocuments• Risk management planFigure 11-3. Plan Risk Management: Data Flow DiagramThe Plan Risk Management process should begin when a project is conceived and should be completed early in the project. It may be necessary to revisit this process later in the project life cycle for example at a major phase change or if the project scope changes significantly or if a subsequent review of risk management effectiveness determines that the Project Risk Management process requires modification.11.1.1 PLAN RISK MANAGEMENT: INPUTS11.1.1.1 PROJECT CHARTERDescribed in Section 4.1.3.1. The project charter documents the high-level project description and boundaries high-level requirements and risks.

### Cleaning Extracted Content
Some Sections end at the same page as another Section begins so we're removing extra contents that doesnt belong to a specific Section [getting the content of each section only when we find the section and the title in the content]

*example : Section '11.1' Title : 'Plan Risk Management' Content : ' • Project   charter11.1 Plan Risk Management'* -> Cleaned Content : '11.1 Plan Risk Management'

In [124]:
def extract_substring(row):
    start_string = row['Section'] + ' ' + row['Title'].upper()
    start_index = row['Content'].find(start_string)
    if start_index != -1:
        return row['Content'][start_index:]
    else:
      pass

In [236]:
# df['Content'] = df.apply(extract_substring, axis=1)
# df.head()

In [237]:
# def verifying_with_all_contents(df):
#     for i, row in df.iterrows():
#         print(f'For the {i}-th Content')
#         title = row['Section'] + ' ' + row['Title'].upper()
#         start = row['Starting_Page_Number']
#         end = row['Ending_Page_Number']
#         print(f'- With the Title = {title}')
#         print(f'Starting in the Page : {start} Ending in the Page : {end}')
#         print('- Content : ')
#         print(row['Extracted_Content'])
#         print('----------------')

# verifying_with_all_contents(df)

### Removing Figures

In [ ]:
# @title Saving Contents in Text files to remove Figures Manually
# output_directory = '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1'
# os.makedirs(output_directory, exist_ok=True)
# def saving_each_extracted_content_in_a_text_file(df):
#   for index, row in df.iterrows():
#     file_name = f"{row['Section']}_{row['Title'].upper()}_{row['Starting_Page_Number']}_{row['Ending_Page_Number']}.txt"
#     file_path = os.path.join(output_directory,file_name)
#     with open(file_path, 'w') as file:
#       file.write(row['Extracted_Content'])
# saving_each_extracted_content_in_a_text_file(df)
# df.drop('Content', axis=1, inplace = True)
# df.drop('Extracted_Content', axis=1, inplace=True)

In [ ]:
# @title cleaning file names to get the content easily
# def remove_digits_from_filenames(directory_path):
#     if not os.path.exists(directory_path):
#         print(f"Directory not found: {directory_path}")
#         return
#     for filename in os.listdir(directory_path):
#         if filename.endswith('.txt'):
#             file_path = os.path.join(directory_path, filename)
#             clean_filename = re.sub(r'\d+\.\d+', '', filename)
#             clean_filename = re.sub(r'\.\d+', '', filename)
#             clean_filename = re.sub(r'_', '', filename)
#             clean_filename = re.sub(r'\d', '', filename)
#             new_file_path = os.path.join(directory_path, clean_filename)
#             os.rename(file_path, new_file_path)
# remove_digits_from_filenames(r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS1')

In [126]:
# @title getting Contents without figures from text files
df['Title'] = df['Title'].str.replace(':','')
def add_clean_content_to_dataframe(df, directory_path):
    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        return
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            title = os.path.splitext(filename)[0]
            matching_row = df[df['Title'].str.lower()  == title.lower()]
            if not matching_row.empty:
                with open(file_path, 'r') as file:
                    content = file.read()
                df.loc[matching_row.index[0], 'Content'] = content
add_clean_content_to_dataframe(df, r'/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_CONTENTS')

In [127]:
df.head()

,Title,Starting_Page_Number,Section,Ending_Page_Number,Content
0,PROJECT RISK MANAGEMENT,401,11,402,Project Risk Management includes the processes...
1,Plan Risk Management,402,11.1,404,Plan Risk Management is the process of definin...
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.1.1 PROJECT CHARTER Described in Section ...
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.2.1 EXPERT JUDGMENT Described in Section...
4,Plan Risk Management Outputs,409,11.1.3,411,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...


### Removing Unecessary Columns

In [129]:
# df.drop(['Starting_Page_Number', 'Ending_Page_Number','Section'], axis=1 , inplace = True)
df.rename(columns={'Content': 'Description'}, inplace=True)
df.rename(columns={'Title':'Concept'}, inplace=True)
# df.drop('Extracted_Content', axis = 1 , inplace = True)
df.head()

,Concept,Starting_Page_Number,Section,Ending_Page_Number,Description
0,PROJECT RISK MANAGEMENT,401,11,402,Project Risk Management includes the processes...
1,Plan Risk Management,402,11.1,404,Plan Risk Management is the process of definin...
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.1.1 PROJECT CHARTER Described in Section ...
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.2.1 EXPERT JUDGMENT Described in Section...
4,Plan Risk Management Outputs,409,11.1.3,411,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...


In [130]:
def clean_text(text):
    # Apply the provided cleaning code
    text = re.sub(r'\be\.g\b', 'for example', text)
    text = re.sub(r',', '', text)
    text = ' '.join([word.lower() for word in text.split() if word.lower() not in stop]) # Lowercase + removing stop words
    return text

In [131]:
df['Cleaned_Description'] = df['Description'].apply(clean_text)
df.head()

,Concept,Starting_Page_Number,Section,Ending_Page_Number,Description,Cleaned_Description
0,PROJECT RISK MANAGEMENT,401,11,402,Project Risk Management includes the processes...,project risk management includes processes con...
1,Plan Risk Management,402,11.1,404,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...
4,Plan Risk Management Outputs,409,11.1.3,411,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...


## Figures Extractions

### Correcting Images Names and Substracting by 36 to get the right Figure's Page Number

In [ ]:
# folder_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmbok_cropped_images"
# image_files = os.listdir(folder_path)
# for filename in image_files:
#     if "_page-" in filename:
#         page_number = filename.split("_page-")[1].split(".")[0] + ".jpg"
#         # Remove leading '0' from the page number
#         page_number = page_number.lstrip('0')
#         new_name = os.path.join(folder_path,page_number)
#         old_path = os.path.join(folder_path, filename)
#         os.rename(old_path, new_name)
#         print(f'Renamed: {filename} to {page_number}')
# subtract_value = 36
# for filename in os.listdir(folder_path):
#     if filename.endswith(".jpg"):
#         file_extension = filename.split('.')[-1]
#         current_number = int(filename.split('.')[0])
#         new_number = current_number - subtract_value
#         new_filename = f"{new_number}.{file_extension}"
#         os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))
#         print(f'Renamed: {filename} to {new_filename}')

In [132]:
folder_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmbok_cropped_images"
image_files = os.listdir(folder_path)
image_numbers = [int(filename.split('.')[0]) for filename in image_files]

In [133]:
df["References"] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    start_page = row["Starting_Page_Number"]
    end_page = row["Ending_Page_Number"]
    references = [os.path.join(folder_path, f'{image_name}.jpg') for image_name in image_numbers if start_page <= image_name < end_page]
    df.at[index, "References"] = references
df.head()

,Concept,Starting_Page_Number,Section,Ending_Page_Number,Description,Cleaned_Description,References
0,PROJECT RISK MANAGEMENT,401,11,402,Project Risk Management includes the processes...,project risk management includes processes con...,[/content/drive/MyDrive/PRM/PRM-Conceptual-Gra...
1,Plan Risk Management,402,11.1,404,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,[/content/drive/MyDrive/PRM/PRM-Conceptual-Gra...
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[]
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,[/content/drive/MyDrive/PRM/PRM-Conceptual-Gra...
4,Plan Risk Management Outputs,409,11.1.3,411,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,[/content/drive/MyDrive/PRM/PRM-Conceptual-Gra...


### Change References to URL's Using ImgBB

In [134]:
with open('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_IMG_BB.txt', 'r') as file:
    imgbb_urls = file.read().splitlines()
reference_to_imgbb = {}
im_num = []
for reference in imgbb_urls:
    image_name = reference.split('/')[-1]
    reference_to_imgbb[image_name] = reference
    im_num.append(int(image_name[:-4]))
print(im_num)

[396, 401, 402, 406, 407, 408, 409, 410, 419, 420, 426, 428, 429, 433, 434, 435, 437, 438, 449, 453, 454]


In [135]:
df["References2"] = [[] for _ in range(len(df))]
image_number_to_url = {}
for reference in imgbb_urls:
    image_name = reference.split('/')[-1]
    image_number = int(image_name.split('.')[0])
    image_number_to_url[image_number] = reference
for index, row in df.iterrows():
    start_page = row["Starting_Page_Number"]
    end_page = row["Ending_Page_Number"]
    references = []
    for image_number in im_num:
        if start_page <= image_number < end_page:
            imgbb_url = image_number_to_url.get(image_number)
            if imgbb_url:
                references.append(imgbb_url)
    df.at[index, "References2"] = references
df.drop('References', axis = 1 , inplace= True)
df.rename(columns={'References2':'References'}, inplace=True)
df.head()

,Concept,Starting_Page_Number,Section,Ending_Page_Number,Description,Cleaned_Description,References
0,PROJECT RISK MANAGEMENT,401,11,402,Project Risk Management includes the processes...,project risk management includes processes con...,[https://i.ibb.co/rssfq3x/401.jpg]
1,Plan Risk Management,402,11.1,404,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,[https://i.ibb.co/fXjdfx5/402.jpg]
2,Plan Risk Management Inputs,404,11.1.1,405,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[]
3,Plan Risk Management Tools and Techniques,405,11.1.2,409,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"[https://i.ibb.co/RH2d7J4/406.png, https://i.i..."
4,Plan Risk Management Outputs,409,11.1.3,411,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"[https://i.ibb.co/47Jwqzc/409.jpg, https://i.i..."


## PMBOK CLEAN DATAFRAME

In [136]:
df.drop(['Starting_Page_Number','Section','Ending_Page_Number'], axis=1, inplace=True)
df.head()

,Concept,Description,Cleaned_Description,References
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,[https://i.ibb.co/rssfq3x/401.jpg]
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,[https://i.ibb.co/fXjdfx5/402.jpg]
2,Plan Risk Management Inputs,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[]
3,Plan Risk Management Tools and Techniques,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"[https://i.ibb.co/RH2d7J4/406.png, https://i.i..."
4,Plan Risk Management Outputs,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"[https://i.ibb.co/47Jwqzc/409.jpg, https://i.i..."


In [137]:
df.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_Clean.csv', index=False)

# Extracting PMI

In [139]:
def extract_pdf(file_path,from_page_number,to_page_number):
    extracted_data = ""
    pdf = PyPDF2.PdfReader(file_path)
    for i in range(from_page_number,to_page_number):
        page = pdf.pages[i]
        extracted_data += page.extract_text()
    return extracted_data

In [140]:
figures_list=extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",10,12)

In [ ]:
print(figures_list)

©2009 Project Management Institute. Practice Standard for Project Risk Management  LIST OF FIGURES  
 Figure 1-1. Hierarchy of PMI Project Risk Management Resources ....................................................2 
 Figure 1-2. Critical Success Factors for Project Risk Management .....................................................6  Figure 3-1. Project Risk Management Process Flow Diagram ...........................................................17  Figure 4-1. Key Areas of Focus for the Plan Risk Management Process ...........................................23  Figure 5-1. Three Perspectives of Risk Identification .........................................................................27  Figure 5-2. Cause, Risk, and Effect ......................................................................................................29  Figure 6-1. Building Risk Analysis Credibility .....................................................................................32 
 Figure 6-2. T

In [141]:
def cleaning_list_of_figures_PMI(list_of_figures):
    # Define a list of patterns and substitutions
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),  # Replace 'fi' with 'fi'
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),  # Remove footer
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),  # Remove the copyright notice
        (r'\.{2,}', ' '),  # Remove points
        (r'[^A-Za-z0-9\s\,\\-]+', ''),  # Remove special characters (keep digits, punctuation, and spaces)
        (r'\s+', ' '),  # Remove extra spaces
        (r'\S*LIST OF FIGURES\S*', ''),  # Remove text and characters attached to it
        (r'\s[0-9]+\s', ' '), #Remove page numbers
        (r'Figure\s([0-9]+)\s*-\s*([0-9]+)', r'Figure \1-\2.'),

    ]

    # Apply all patterns and substitutions in sequence
    for pattern, substitution in patterns_substitutions:
        list_of_figures = re.sub(pattern, substitution, list_of_figures)
    return list_of_figures

In [142]:
clean_lof=cleaning_list_of_figures_PMI(figures_list)
clean_lof

'  Figure 1-1. Hierarchy of PMI Project Risk Management Resources Figure 1-2. Critical Success Factors for Project Risk Management Figure 3-1. Project Risk Management Process Flow Diagram Figure 4-1. Key Areas of Focus for the Plan Risk Management Process Figure 5-1. Three Perspectives of Risk Identification Figure 5-2. Cause, Risk, and Effect Figure 6-1. Building Risk Analysis Credibility Figure 6-2. The Perform Qualitative Risk Analysis Process Figure 7-1. Comparison of Qualitative and Quantitative Approaches Figure 7-2. Structure of a Quantitative Risk Analysis Figure 8-1. Critical Success Factors for Risk Response Planning Figure 8-2. The Steps Involved in Planning Risk Responses Figure 9-1. Schematic Representation of the Monitor and Control Risks Process xi'

In [143]:
def create_figures_list(text):
    figure_pattern = re.compile(r'Figure\s')
    text_with_delimiter = re.sub(figure_pattern, r'\\\\\g<0>', text)
    figures = text_with_delimiter.split('\\')
    figures = [figure.strip() for figure in figures if figure.strip()]
    return figures

In [144]:
figures = create_figures_list(clean_lof)
len(figures)

13

In [145]:
def extract_images_from_pdf(file_path, figures):
    output_folder = "figures"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    pdf_file = fitz.open(file_path)
    i = 0
    j = 1
    for page_index in range(len(pdf_file)):
        page = pdf_file[page_index]
        image_list = page.get_images(full=True)
        if image_list:
            print(f"[+] Found a total of {len(image_list)} images in page {page_index}")

        for image_index, img in enumerate(image_list, start=1):
            xref = img[0]
            base_image = pdf_file.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            pil_image = Image.open(io.BytesIO(image_bytes))
            if i < len(figures):
                image_filename = os.path.join(output_folder, f"{figures[i]}.{image_ext}")
                i = i + 1
            else:
                image_filename = os.path.join(output_folder, f"Figure D{j}.{image_ext}")
                j = j + 1
            pil_image.save(image_filename)

            print(f"    Image {image_index} saved as {image_filename}")
    pdf_file.close()
    return

In [146]:
extract_images_from_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",figures)

[+] Found a total of 1 images in page 13
    Image 1 saved as figures/Figure 1-1. Hierarchy of PMI Project Risk Management Resources.png
[+] Found a total of 1 images in page 17
    Image 1 saved as figures/Figure 1-2. Critical Success Factors for Project Risk Management.png
[+] Found a total of 1 images in page 28
    Image 1 saved as figures/Figure 3-1. Project Risk Management Process Flow Diagram.png
[+] Found a total of 1 images in page 34
    Image 1 saved as figures/Figure 4-1. Key Areas of Focus for the Plan Risk Management Process.png
[+] Found a total of 1 images in page 38
    Image 1 saved as figures/Figure 5-1. Three Perspectives of Risk Identification.png
[+] Found a total of 1 images in page 40
    Image 1 saved as figures/Figure 5-2. Cause, Risk, and Effect.png
[+] Found a total of 1 images in page 43
    Image 1 saved as figures/Figure 6-1. Building Risk Analysis Credibility.png
[+] Found a total of 1 images in page 44
    Image 1 saved as figures/Figure 6-2. The Perfor

## Loading the PMI Corpus

In [147]:
extracted_data_PMI = extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",12,123)

In [148]:
print(extracted_data_PMI[98:3000])

INTRODUCTION 
 Project Management Institute (PMI) practice standards are guides to the use of a tool, technique, or process 
identiﬁ  ed in  A Guide to the Project Management Body of Knowledge   ( PMBOK   ®  Guide  – Fourth Edition) or 
other PMI standards. Practice standards are targeted at audiences who participate in the management of 
projects. This includes project managers, project personnel, contract personnel, supervisors, and other project stakeholders. 
 A PMI practice standard describes processes, activities, inputs, and outputs for a speciﬁ  c Knowledge Area. 
It provides information on what the signiﬁ  cant process, tool, or technique is, what it does, why it is signiﬁ  cant, 
when it should be performed or executed, and, if necessary for further clariﬁ  cation, who should perform the process. A practice standard does not prescribe how the process is to be implemented, leaving that subject for other forums such as handbooks, manuals, and courses. 
 This chapter includes th

### PMI CLEANING

In [149]:
def cleaning_pmi(pmi_corpus):
    # Define a list of patterns and substitutions
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),  # Replace 'fi' with 'fi'
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),  # Remove footer
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),  # Remove the copyright notice
        (r'[^A-Za-z0-9\s\.,-]+', ''),  # Remove special characters (keep digits, punctuation, and spaces)
        (r'\s+', ' '),
        (r'This chapter includes(.*?Factors for Project Risk Management)',''),
        (r'Agreed-Upon', r'Agreed Upon'),
        (r'High-Quality', r'High Quality'),
        (r'Risk-Related', r'Risk Related'),
        (r'In uence', r'Influence'),
        (r'in uence', r'influence'),
        (r'\s+', ' '),  # Remove extra spaces
    ]

    # Apply all patterns and substitutions in sequence
    for pattern, substitution in patterns_substitutions:
        pmi_corpus = re.sub(pattern, substitution, pmi_corpus)
    # Remove the titles of figures from the text
    for figure in figures:
        pmi_corpus = re.sub(r'\s' + re.escape(figure) + r'\s', ' ', pmi_corpus)
    return pmi_corpus

In [150]:
clean_pmi = cleaning_pmi(extracted_data_PMI)
clean_pmi[98:2000]

'of a tool, technique, or process identified in A Guide to the Project Management Body of Knowledge PMBOK Guide Fourth Edition or other PMI standards. Practice standards are targeted at audiences who participate in the management of projects. This includes project managers, project personnel, contract personnel, supervisors, and other project stakeholders. A PMI practice standard describes processes, activities, inputs, and outputs for a specific Knowledge Area. It provides information on what the significant process, tool, or technique is, what it does, why it is significant, when it should be performed or executed, and, if necessary for further clarification, who should perform the process. A practice standard does not prescribe how the process is to be implemented, leaving that subject for other forums such as handbooks, manuals, and courses. CHAPTER 1 INTRODUCTION 1.1 Purpose of the Practice Standard for Project Risk Management The purpose of the Practice Standard for Project Risk 

In [151]:
def extract_subsubsection_titles(text):
    patterns_to_remove = [
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),  # Remove footer
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),
        ('–', ''),
        ('  ', ' '),
        ('“|”', ''),
        (r'Inﬂ uence','Influence'),
        (r'[()/]', ''),  # Remove both parentheses and forward slash
    ]

    for pattern, replacement in patterns_to_remove:
        text = re.sub(pattern, replacement, text)

    pattern = re.compile(r'[0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+')
    matching_lines = [line.strip() for line in text.split('\n') if pattern.search(line)]
    return matching_lines
# Assuming extracted_data_PMI is defined elsewhere
subsubtitles = extract_subsubsection_titles(extracted_data_PMI)
print(subsubtitles)

['D.1.1.1 The Risk Management Plan Purpose and Typical Content', 'D.1.1.2 Templates Relevant to Risk Management Planning', 'D.1.1.3 Risk Statement Metalanguage', 'D.1.1.4 Risk Prioritization and Selection Guidelines', 'Examples of techniques are given in D.2.1.1 through D.2.1.15.', 'D.2.1.1 Assumptions and Constraints Analysis', 'D.2.1.2 Brainstorming', 'D.2.1.3 Cause and Effect Ishikawa Diagrams', 'D.2.1.4 Checklists', 'D.2.1.5 Delphi Technique', 'D.2.1.6 Document Review', 'D.2.1.7 FMEAFault Tree Analysis', 'D.2.1.8 Force Field Analysis', 'D.2.1.9 Industry Knowledge Base', 'D.2.1.10 Influence Diagrams', 'D.2.1.11 Interviews', 'D.2.1.12 Nominal Group Technique', 'D.2.1.13 Post-Project ReviewsLessons LearnedHistorical Information', 'D.2.1.14 Prompt Lists', 'prompt lists have been developed for use in risk identiﬁ cation, and some of these are presented below. These can then be used as a framework for other risk identiﬁ cation techniques such as brainstorming see D.2.1.2 or risk intervie

### Extract table of contents from PMI

In [152]:
table_of_contents_PMI=extract_pdf("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/input/practice-standard-project-risk-management.pdf",4,9)
print(table_of_contents_PMI[98:2000])


 CHAPTER 1  -  INTRODUCTION  .................................................................................................. .1
 1.1 Purpose of the Practice Standard for Project Risk Management  .......................... 2
 1.2 Project Risk Management Deﬁ  nition  ...................................................................... 4
 1.3 Role of Project Risk Management in Project Management  ................................... 4
 1.4 Good Risk Management Practice  ............................................................................ 5 1.5 Critical Success Factors for Project Risk Management  ......................................... 6
 1.6 Conclusion  .............................................................................................................. .7
 CHAPTER 2  -  PRINCIPLES AND CONCEPTS ..............................................................................9
 2.1 Introduction  .........................................................................

### Cleaning the table of contents

In [153]:
def cleaning_table_of_contents_PMI(table_of_contents_PMI):
    # Define a list of patterns and substitutions
    patterns_substitutions = [
        (r'ﬁ\s\s', 'fi'),  # Replace 'fi' with 'fi'
        (r'APPENDIX C', '\n APPENDIX C'),
        (r'\d©2009 Project Management Institute\. Practice Standard for Project Risk Management\d', ''),  # Remove footer
        (r'©2009 Project Management Institute. Practice Standard for Project Risk Management', ''),  # Remove the copyright notice
        (r'\.{2,}', ' '),  # Remove points
        (r'\s.\d\s', ' \ '),  # Remove remaining points
        (r'60B.3', r' \\ B.3'),
        (r'-', r' '),
        (r'[^A-Za-z0-9\s\.,\\]+', ''),  # Remove special characters (keep digits, punctuation, and spaces)
        (r'(?<=\s)([0-9]+)(?=\s)', r'\1 \\ '),  # Insert backslash after digits between spaces
        (r'\s+', ' '),  # Remove extra spaces
        (r'\S*TABLE OF CONTENTS\S*', ''),  # Remove text and characters attached to it
        (r'(?<!CHAPTER) (\d+) \\ ', '  '),  # Remove page numbers
        (r'CHAPTER (\d+) \\ ', r'CHAPTER \1 '),  #Remove the backslash after the digits in the "CHAPTER" string
        (r'PreProject', r'Pre-Project'),
    ]
    # Apply all patterns and substitutions in sequence
    for pattern, substitution in patterns_substitutions:
        table_of_contents_PMI = re.sub(pattern, substitution, table_of_contents_PMI)
    # Define the patterns to match
    #patterns_to_remove = [re.compile(r'^\d+\.\d+\.\d+'), re.compile(r'^[A-Z]\.\d+\.\d+')]
    return table_of_contents_PMI


In [154]:
clean_toc_PMI=cleaning_table_of_contents_PMI(table_of_contents_PMI)
clean_toc_PMI[98:2000]

'roject Risk Management Definition \\ 1.3 Role of Project Risk Management in Project Management \\ 1.4 Good Risk Management Practice \\ 1.5 Critical Success Factors for Project Risk Management \\ 1.6 Conclusion \\ CHAPTER 2 PRINCIPLES AND CONCEPTS \\ 2.1 Introduction \\ 2.2 Definition of Project Risk \\ 2.3 Individual Risks and Overall Project Risk \\ 2.4 Stakeholder Risk Attitudes \\ 2.5 Iterative Process \\ 2.6 Communication \\ 2.7 Responsibility for Project Risk Management \\ 2.8 Project Managers Role for Project Risk Management \\ CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEMENT PROCESSES \\ 3.1 Project Risk Management and Project Management \\ 3.2 Project Risk Management Processes \\ CHAPTER 4 PLAN RISK MANAGEMENT \\ 4.1 Purpose and Objectives of the Plan Risk Management Process \\ 4.2 Critical Success Factors for the Plan Risk Management Process \\ 4.2.1 Identify and Address Barriers to Successful Project Risk Management \\ 4.2.2 Involve Project Stakeholders in Project Risk Mana

### Segment PMI into chapters

In [155]:
def segment_book_into_chapters(book_text):
    # Define the pattern for chapter headings (assuming they start with "CHAPTER X - TITLE")
    pattern = re.compile(r'CHAPTER \d [A-Z ]+ |APPENDIX [A-Z, ]+ |GLOSSARY\s')
    # Find all occurrences of the pattern in the text
    chapter_headings = re.findall(pattern, book_text)
    # Split the book into chapters using the found headings
    chapter_segments = re.split(pattern, book_text)
    # Initialize an empty DataFrame
    columns = ['Chapter', 'Content']
    df = pd.DataFrame(columns=columns)
    for heading, content in zip(chapter_headings, chapter_segments[1:]):
        heading = heading.strip().upper()  # Convert heading to uppercase for consistency
        content = content.strip()
        # Check if the heading partially matches with existing chapters
        matching_chapters = [existing_heading for existing_heading in df['Chapter'] if heading in existing_heading]
        if matching_chapters:
            # If partial match exists, find the index and concatenate the content
            idx = df.index[df['Chapter'] == matching_chapters[0]].tolist()[0]
            df.at[idx, 'Content'] += ' ' + content
        else:
            # If it's a new heading, add it to the DataFrame
            df = df.append({'Chapter': heading, 'Content': content}, ignore_index=True)
    return df

In [156]:
#Segment PMI into chapters
chapters_data = segment_book_into_chapters(clean_pmi)
chapters_data.head(5)

,Chapter,Content
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,CHAPTER 2 PRINCIPLES AND CONCEPTS,2.1 Introduction This chapter introduces the k...
2,CHAPTER 3 INTRODUCTION TO PROJECT RISK MANAGEM...,3.1 Project Risk Management and Project Manage...
3,CHAPTER 4 PLAN RISK MANAGEMENT,4.1 Purpose and Objectives of the Plan Risk Ma...
4,CHAPTER 5 IDENTIFY RISKS,5.1 Purpose and Objectives of the Identify Ris...


### Extract the chapters from the table of contents

In [157]:
def segment_toc_into_chapters(book_text):
    # Define the pattern for chapter headings (assuming they start with "CHAPTER X - TITLE")
    pattern = re.compile(r'CHAPTER \d [A-Z ]+ |APPENDIX [A-Z ] ')
    # Find all occurrences of the pattern in the text
    chapter_headings = re.findall(pattern, book_text)
    # Split the book into chapters using the found headings
    chapter_segments = re.split(pattern, book_text)
    # Combine the headings and content into a dictionary
    chapters_data = {}
    for heading, content in zip(chapter_headings, chapter_segments[1:]):
        heading = heading.strip()
        content = content.strip()
        key = heading.upper()  # Convert heading to uppercase for consistency
        if key in chapters_data:
            # If the heading already exists, combine the content with a single space
            chapters_data[key] += ' ' + content
        else:
            # If it's a new heading, add it to the dictionary
            chapters_data[key] = content
    return chapters_data

In [158]:
toc_data = segment_toc_into_chapters(clean_toc_PMI)

### Extract all the titles from the table of contents

In [159]:
def Extract_title_and_subtitles_toc(chapters_data):
    updated_chapters_data = {}
    for chapter, content in chapters_data.items():
        # Split the chapter into sections based on backslashes
        sections = content.split(' \\ ')
        # Remove empty sections and remaining \\
        sections = [section.strip() for section in sections if section.strip()]
        sections = [re.sub(r'\\', '', section) for section in sections if section.strip()]
        # Update the chapters data with sections
        updated_chapters_data[chapter] = sections
    return updated_chapters_data

In [160]:
updated_toc_data = Extract_title_and_subtitles_toc(toc_data)

### Extract only the section titles from the table of contents

In [161]:
# Define a pattern to match section titles like '1.1', '2.2', etc.
section_pattern = re.compile(r'\s*([0-9]\.[0-9]|[A-Z]\.[0-9])\s')
# Create a new dictionary to store chapter-wise section titles
chapter_sections = {}
# Iterate through each chapter in the updated_toc_data dictionary
for chapter, sections in updated_toc_data.items():
    # Filter sections that match the section title pattern
    section_titles = [section.strip() for section in sections if section_pattern.match(section)]
    # Add the chapter and its corresponding section titles to the new dictionary
    chapter_sections[chapter] = section_titles
    # Check if the chapter is 'APPENDIX B' and add a specific section
    if chapter == 'APPENDIX B':
        chapter_sections[chapter].append('B.4 Exposure and Consensus')

### Segment each chapter into sections

In [162]:
def segment_chapter_into_sections(chapters_data):
    # Initialize an empty dictionary to store updated chapters data
    updated_chapters_data = {}
    # Iterate through each row in the DataFrame
    for index, row in chapters_data.iterrows():
        chapter = row['Chapter']
        content = row['Content']
        # Add backslashes to numbers
        updated_content = re.sub(r'\s((?!8\.0)[0-9]\.[0-9]|[A-Z]\.[0-9])\s', r' \\ \1 ', content)
        # Split the chapter into sections based on backslashes
        sections = updated_content.split(' \\ ')
        # Remove empty sections
        sections = [section.strip() for section in sections if section.strip()]
        # Update the chapters data with sections
        updated_chapters_data[chapter] = sections
    # Convert the dictionary to a list of dictionaries for DataFrame creation
    data_for_df = [{'Chapter': chapter, 'Section': section} for chapter, sections in updated_chapters_data.items() for section in sections]
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data_for_df)
    return df

In [163]:
updated_chapters_data = segment_chapter_into_sections(chapters_data)
updated_chapters_data.head(5)

,Chapter,Section
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition The def...
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice Project Risk...


### Create the Section_title column

In [164]:
# Create a new column 'section_title' in the DataFrame
updated_chapters_data['Section_title'] = None
# Iterate through each row in the DataFrame
for index, row in updated_chapters_data.iterrows():
    section_title = ''
    # Iterate through each chapter in the dictionary
    for chapter, section_titles in chapter_sections.items():
        # Check if the row's section matches any title in the current chapter
        for title in section_titles:
            if title.strip().lower() in row['Section'].strip().lower():
                section_title = title.strip()
                # Remove the matched title from the 'Section' column
                updated_chapters_data.at[index, 'Section'] = row['Section'].replace(title, '').strip()
                break
        if section_title:
            break
    # Assign the matched section title to the 'section_title' column
    updated_chapters_data.at[index, 'Section_title'] = section_title
# Reorder the columns
updated_chapters_data = updated_chapters_data[['Chapter', 'Section_title', 'Section']]
# Print the updated DataFrame
updated_chapters_data

,Chapter,Section_title,Section
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...
...,...,...,...
59,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.4 Techniques, Examples and Templates for Per..."
60,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...","D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...
61,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon..."
62,APPENDIX E REFERENCES,,Association for Project Management. 2004. Proj...


### Extract sub-titles from the table of contents

In [165]:
# Define a pattern to match section titles like '1.1', '2.2', etc.
subsection_pattern = re.compile(r'\s*([0-9]\.[0-9]\.[0-9]+|[A-Z]\.[0-9]\.[0-9]+)\s')
# Create a new dictionary to store chapter-wise section titles
chapter_subsections = {}
# Iterate through each chapter in the updated_toc_data dictionary
for chapter, subsections in updated_toc_data.items():
    # Filter sections that match the section title pattern
    subsection_titles = [subsection.strip() for subsection in subsections if subsection_pattern.match(subsection)]
    # Add the chapter and its corresponding section titles to the new dictionary
    chapter_subsections[chapter] = subsection_titles

### Segment section into subsections

In [166]:
def segment_sections_into_subsections(updated_clean_pmi):
    # Initialize an empty list to store data for the new DataFrame
    data_for_df = []
    # Initialize variables to keep track of current chapter and section title
    current_chapter = None
    current_section_title = None
    current_section = None
    # Iterate through each row in the DataFrame
    for index, row in updated_clean_pmi.iterrows():
        chapter = row['Chapter']
        section_title = row['Section_title']
        section = row['Section']
        # Check if the section contains the specified pattern
        matches = list(re.finditer(r'(?<!Sections)(?<!Section)(?<!through)(?<!and)\s((?!7\.3\.8)[0-9]\.[0-9]\.[0-9]+|[A-Z]\.[0-9]\.[0-9]+)\s| 4.3.1 ', section))
        # Check if there are any matches
        if matches:
            # Iterate through all matches in the section
            for match in matches:
                # Find the index of the current title matching the pattern
                title_index = match.start()
                # Extract the part of the section after the current title
                section_after_title = section[title_index:]
                # Add backslashes to subsection numbers
                updated_section = re.sub(r'(?<!Sections)(?<!Section)(?<!through)(?<!and)\s((?!7\.3\.8)[0-9]\.[0-9]\.[0-9]+|[A-Z]\.[0-9]\.[0-9]+)\s| 4.3.1', r' \\ \1 ', section_after_title)
                # Split the section into subsections based on backslashes
                subsections = updated_section.split(' \\ ')
                # If there are subsections, update current_chapter and current_section_title
                if subsections:
                    current_chapter = chapter
                    current_section_title = section_title
                # Remove identified subsections from the original section
                for subsection in subsections:
                    section = section.replace(subsection, '').strip()
                # Append data for the new DataFrame for each subsection
                for subsection in subsections:
                    # Check if the section starts with '4.3.1'
                    if section.startswith('4.3.1') and subsection=='':
                        # If it does, set the content of the section as the Subsection
                        subsection = section
                        data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': '', 'Subsection': subsection})
                    else:
                        data_for_df.append({'Chapter': current_chapter, 'Section_title': current_section_title, 'Section': section, 'Subsection': subsection})
        else:
            # If there are no matches, add the original section with an empty Subsection
            data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section, 'Subsection': ''})
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data_for_df)
    # Remove duplicated rows
    df = df.drop_duplicates()
    # Check conditions for '4.3.1' in the beginning of the Section column
    mask1 = (df['Section'].str.startswith('4.3.1')) & (df['Subsection'] == '')
    mask2 = (df['Section'].str.startswith('4.3.1')) & (df['Subsection'] != '')

    # Update Section and Subsection columns based on conditions
    df.loc[mask1, 'Section'] = ''
    df.loc[mask1, 'Subsection'] = df['Section']

    df.loc[mask2, 'Section'] = ''
    return df

In [167]:
data_for_df = segment_sections_into_subsections(updated_chapters_data)
data_for_df

,Chapter,Section_title,Section,Subsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,
...,...,...,...,...
185,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...","D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...,D.5.1 Techniques for Plan Risk Response Techni...
186,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",
187,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...
188,APPENDIX E REFERENCES,,Association for Project Management. 2004. Proj...,


### Create the Subsection_title column

In [168]:
# Create a new column 'section_title' in the DataFrame
data_for_df['Subsection_title'] = None

# Iterate through each row in the DataFrame
for index, row in data_for_df.iterrows():
    subsection_title = ''
    # Iterate through each chapter in the dictionary
    for chapter, subsection_titles in chapter_subsections.items():
        # Check if the row's section matches any title in the current chapter
        for subtitle in subsection_titles:
            if subtitle.strip().lower() in row['Subsection'].strip().lower():
                subsection_title = subtitle.strip()
                # Remove the matched title from the 'Section' column
                data_for_df.at[index, 'Subsection'] = row['Subsection'].replace(subtitle, '').strip()
                break
        if section_title:
            break

    # Assign the matched section title to the 'section_title' column
    data_for_df.at[index, 'Subsection_title'] = subsection_title

# Reorder the columns
data_for_df = data_for_df[['Chapter', 'Section_title', 'Section','Subsection_title','Subsection']]

# Print the updated DataFrame
data_for_df

,Chapter,Section_title,Section,Subsection_title,Subsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,,
...,...,...,...,...,...
185,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...","D.5 Techniques, Examples, and Templates for Pl...",8 Plan Risk Responses develops the set of acti...,D.5.1 Techniques for Plan Risk Response,Techniques for Plan Risk Responses are given i...
186,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",,
187,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...
188,APPENDIX E REFERENCES,,Association for Project Management. 2004. Proj...,,


### Segment subsections into subsubsections

In [169]:
def segment_subsections_into_subsubsections(updated_clean_pmi):
    # Initialize an empty list to store data for the new DataFrame
    data_for_df = []

    # Iterate through each row in the DataFrame
    for index, row in updated_clean_pmi.iterrows():
        chapter = row['Chapter']
        section_title = row['Section_title']
        section = row['Section']
        subsection_title = row['Subsection_title']
        subsection = row['Subsection']

        # Check if the subsection contains the specified pattern
        matches = list(re.finditer(r'(?<!Sections)(?<!in)(?<!through)(?<!and)(?<!see)\s([0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+)\s', subsection))

        # Check if there are any matches
        if matches:
            # Iterate through all matches in the subsection
            for match in matches:
                # Find the index of the current title matching the pattern
                title_index = match.start()

                # Check if the title is preceded by certainn
                #if title_index > 2 and (subsection[title_index - 2:title_index].lower() == 'and' or subsection[title_index - 2:title_index].lower() == 'in'   or subsection[title_index - 2:title_index].lower() == 'through'  or subsection[title_index - 2:title_index].lower() == 'sections'):
                    #continue


                # Extract the part of the subsection after the current title
                subsection_after_title = subsection[title_index:]

                # Add backslashes to subsubsection numbers
                updated_subsection = re.sub(r'(?<!Sections)(?<!in)(?<!through)(?<!and)(?<!see)\s([0-9]\.[0-9]\.[0-9]\.[0-9]|[A-Z]\.[0-9]\.[0-9]\.[0-9]+)\s', r' \\ \1 ', subsection_after_title)

                # Split the subsection into subsubsections based on backslashes
                subsubsections = updated_subsection.split(' \\ ')

                # Remove empty subsubsections
                subsubsections = [subsubsection.strip() for subsubsection in subsubsections if subsubsection.strip()]
                # Remove the extracted subsections from the original section
                subsection = subsection.replace(subsection_after_title, '')

                # Append data for the new DataFrame for each subsubsection
                if subsubsections:
                    for subsubsection in subsubsections:
                        data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':subsubsection})
                else:
                    # If there are no subsubsections, add the original subsection with an empty Subsubsection
                    data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':''})
        else:
            # If there are no matches, add the original subsection with an empty Subsubsection
            data_for_df.append({'Chapter': chapter, 'Section_title': section_title, 'Section': section,'Subsection_title': subsection_title, 'Subsection': subsection, 'Subsubsection':''})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data_for_df)
    # Remove duplicated rows
    df = df.drop_duplicates()

    return df

In [170]:
updated_data_for_df = segment_subsections_into_subsubsections(data_for_df)
updated_data_for_df

,Chapter,Section_title,Section,Subsection_title,Subsection,Subsubsection
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,,
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,,,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,,,
...,...,...,...,...,...,...
202,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,D.6.1.5 Trend Analysis The evolution of the va...
203,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,D.6.1.6 Variance Analysis The formulae in Earn...
204,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,
209,APPENDIX E REFERENCES,,Association for Project Management. 2004. Proj...,,,


In [171]:
def move_subtitles(df, subsubtitles):
    # Create Subsubsection_title column if it doesn't exist
    if 'Subsubsection_title' not in df.columns:
        df['Subsubsection_title'] = ''

    for subtitle in subsubtitles:
        df['Subsubsection_title'] = df.apply(
            lambda row: subtitle if subtitle in row['Subsubsection'] else row['Subsubsection_title'],
            axis=1
        )
        df['Subsubsection'] = df['Subsubsection'].apply(
            lambda text: text.replace(subtitle, '') if subtitle in text else text
        )

    return df


# Example usage:
# Assuming df is your dataframe and subsubtitles is your list of texts
updated_data_for_df = move_subtitles(updated_data_for_df, subsubtitles)
updated_data_for_df

,Chapter,Section_title,Section,Subsection_title,Subsection,Subsubsection,Subsubsection_title
0,CHAPTER 1 INTRODUCTION,,Project Management Institute PMI practice stan...,,,,
1,CHAPTER 1 INTRODUCTION,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...,,,,
2,CHAPTER 1 INTRODUCTION,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ...",,,,
3,CHAPTER 1 INTRODUCTION,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...,,,,
4,CHAPTER 1 INTRODUCTION,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...,,,,
...,...,...,...,...,...,...,...
202,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,The evolution of the variance values over tim...,D.6.1.5 Trend Analysis
203,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,"The formulae in Earned Value Analysis CV, SV,...",D.6.1.6 Variance Analysis
204,"APPENDIX D TOOLS, TECHNIQUES AND TEMPLATES FOR...",,"D.6 Techniques, Examples and Templates for Mon...",D.6.1 Techniques for Monitor and Control Risks...,Techniques for Monitor and Control Risks are g...,,
209,APPENDIX E REFERENCES,,Association for Project Management. 2004. Proj...,,,,


In [172]:
def process_dataframe(input_df):

    # Remove duplicate rows

    new_df = pd.DataFrame(columns=['Concept','Description'])
    input_df = input_df.drop_duplicates()

    # Create a dictionary to store the mapping of chapters to titles
    chapter_to_title = {}
    section_title_seen = set()
    subsection_title_seen = set()

    # Iterate through the DataFrame
    for index, row in input_df.iterrows():
        chapter = row['Chapter']
        section = row['Section']
        subsection = row['Subsection']
        subsubsection = row['Subsubsection']
        section_title = row['Section_title']
        subsection_title = row['Subsection_title']
        subsubsection_title = row['Subsubsection_title']


        # Check if the chapter has already been encountered
        if chapter in chapter_to_title:

            if section_title in section_title_seen:

                if subsection_title in subsection_title_seen:
                    title = subsubsection_title
                    description=subsubsection

                elif subsection_title not in subsection_title_seen:
                    title = subsection_title
                    subsection_title_seen.add(subsection_title)
                    description=subsection

            elif section_title not in section_title_seen:
                title = section_title
                section_title_seen.add(section_title)
                description=section
        else:
            # If not, use the Chapter as the Title and update the mapping
            title = chapter
            chapter_to_title[chapter] = title
            description=section

        # Add the Title to the new DataFrame
        new_df = new_df.append({'Concept': title,'Description':description}, ignore_index=True)
        # Check if the subsubsection_title follows the specified pattern
        pattern = re.compile(r'^D\.\d+\.\d+\.1\s')

        if pattern.match(subsubsection_title):
            new_df = new_df.append({'Concept': subsubsection_title, 'Description': subsubsection}, ignore_index=True)

        # Replace empty strings with NaN and then drop rows with any NaN values
        new_df = new_df.replace('', pd.NA).dropna()




    return new_df

In [173]:
df = process_dataframe(updated_data_for_df)
df

,Concept,Description
0,CHAPTER 1 INTRODUCTION,Project Management Institute PMI practice stan...
1,1.1 Purpose of the Practice Standard for Proje...,The purpose of the Practice Standard for Proje...
2,1.2 Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,1.3 Role of Project Risk Management in Project...,Project Risk Management is not an optional act...
4,1.4 Good Risk Management Practice,Project Risk Management is a valuable componen...
...,...,...
171,D.6.1.4 Status Meetings,Risks can and should be on the agenda at all ...
172,D.6.1.5 Trend Analysis,The evolution of the variance values over tim...
173,D.6.1.6 Variance Analysis,"The formulae in Earned Value Analysis CV, SV,..."
174,APPENDIX E REFERENCES,Association for Project Management. 2004. Proj...


### Extract concepts from the titles

In [175]:
def extract_concepts_from_titles(df):
    #stop_words = set(stopwords.words('english'))
    for index, row in df.iterrows():

        # Clean chapter titles
        chapter_header = r'(CHAPTER (\d)|APPENDIX [A-Z])+'
        row['Concept'] = re.sub(chapter_header, '', row['Concept'])

        # Clean section titles
        section_header = r'([1-9]\.[1-9]+|[A-Z]\.[1-9]+)'
        row['Concept'] = re.sub(section_header, '', row['Concept'])

        # Clean subsection titles
        subsection_header = r'([1-9]\.[1-9]\.[1-9]+|[A-Z]\.[1-9]\.[1-9]+)'
        row['Concept'] = re.sub(subsection_header, '', row['Concept'])

        # Clean subsubsection titles
        subsubsection_header = r'([1-9]\.[1-9]\.[1-9]\.[1-9]+|[A-Z]\.[1-9]\.[1-9]\.[1-9]+)'
        row['Concept'] = re.sub(subsubsection_header, '', row['Concept'])

        # Convert to lowercase
        #row['Concept'] = row['Concept'].lower()

        # Remove stopwords
        #words = row['Concept'].split()
        #filtered_words = [word for word in words if word.lower() not in stop_words]
        #row['Concept'] = ' '.join(filtered_words)

    return df

In [176]:
data=extract_concepts_from_titles(df)
data.head(20)

,Concept,Description
0,INTRODUCTION,Project Management Institute PMI practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...
5,Critical Success Factors for Project Risk Man...,Specific criteria for success of each Project ...
6,Conclusion,The principles of Project Risk Management desc...
7,PRINCIPLES AND CONCEPTS,This chapter introduces the key ideas required...
8,Definition of Project Risk,The word risk is used in many ways in everyday...
9,Individual Risks and Overall Project Risk,It is useful to consider project risk at two l...


In [177]:
data['Concept'] = data['Concept'].str.replace(r'\.\d+', '', regex=True)
data['Concept'] = data['Concept'].str.replace(r'^.\s', '', regex=True)

In [178]:
data.head(5)

,Concept,Description
0,INTRODUCTION,Project Management Institute PMI practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ..."
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...


In [179]:
data['Cleaned_Description'] = data['Description'].apply(clean_text)

In [180]:
data.head(5)

,Concept,Description,Cleaned_Description
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...


### Extracting References PMI

In [181]:
figures = create_figures_list(clean_lof)
j=1
for i in range(1, 21):
    if i==1:
        figures.append(f"Table D{j}")
        j+=1
        figures.append(f"Table D{j}-1")
        figures.append(f"Table D{j}-2")
        figures.append(f"Table D{j}-3")
        figures.append(f"Table D{j}-4")
        figures.append(f"Table D{j}-5")
        j+=1
    if i in (11,14,18,20):
        if i==11:
            figures.append(f"Table D{j}-1")
            figures.append(f"Table D{j}-2")
            figures.append(f"Table D{j}-3")
            j+=1
        if i==14:
            figures.append(f"Table D{j}-1")
            figures.append(f"Table D{j}-2")
            figures.append(f"Table D{j}-3")
            j+=1
        if i==18:
            figures.append(f"Table D{j}-1")
            figures.append(f"Table D{j}-2")
            figures.append(f"Table D{j}-3")
            figures.append(f"Table D{j}-4")
            j+=1
        if i==20:
            figures.append(f"Table D{j}-1")
            figures.append(f"Table D{j}-2")
            figures.append(f"Table D{j}-3")
    figure_name = f"Figure D{i}"
    figures.append(figure_name)

figures

['Figure 1-1. Hierarchy of PMI Project Risk Management Resources',
 'Figure 1-2. Critical Success Factors for Project Risk Management',
 'Figure 3-1. Project Risk Management Process Flow Diagram',
 'Figure 4-1. Key Areas of Focus for the Plan Risk Management Process',
 'Figure 5-1. Three Perspectives of Risk Identification',
 'Figure 5-2. Cause, Risk, and Effect',
 'Figure 6-1. Building Risk Analysis Credibility',
 'Figure 6-2. The Perform Qualitative Risk Analysis Process',
 'Figure 7-1. Comparison of Qualitative and Quantitative Approaches',
 'Figure 7-2. Structure of a Quantitative Risk Analysis',
 'Figure 8-1. Critical Success Factors for Risk Response Planning',
 'Figure 8-2. The Steps Involved in Planning Risk Responses',
 'Figure 9-1. Schematic Representation of the Monitor and Control Risks Process xi',
 'Table D1',
 'Table D2-1',
 'Table D2-2',
 'Table D2-3',
 'Table D2-4',
 'Table D2-5',
 'Figure D1',
 'Figure D2',
 'Figure D3',
 'Figure D4',
 'Figure D5',
 'Figure D6',
 'Fig

### Figure Name Mapping: PMI to Saved Names

In [182]:
repo_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images"
full_path = "/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images"
image_files = os.listdir(repo_path)
sorted_files = sorted(image_files, key=lambda x: (
    int(x.replace("page_2_", "page_").split('_')[1].split('.')[0]),
    int(x.split('_')[-2].split('.')[0])
))
# Create a dictionary with keys from the figures list and values from the sorted files
figures_dict = {figure: sorted_files[i] for i, figure in enumerate(figures)}
figure_d17_key = next(key for key in figures_dict.keys() if key.startswith("Figure D17"))
table_5_1_key = next(key for key in figures_dict.keys() if key.startswith("Table D5-1"))
figure_d19_key = next(key for key in figures_dict.keys() if key.startswith("Figure D19"))
table_6_1_key = next(key for key in figures_dict.keys() if key.startswith("Table D6-1"))
figure_d1_key = next(key for key in figures_dict.keys() if key.startswith("Figure D1"))
figure_d2_key = next(key for key in figures_dict.keys() if key.startswith("Figure D2"))
figures_dict[figure_d17_key], figures_dict[table_5_1_key] = figures_dict[table_5_1_key], figures_dict[figure_d17_key]
figures_dict[figure_d19_key], figures_dict[table_6_1_key] = figures_dict[table_6_1_key], figures_dict[figure_d19_key]
figures_dict[figure_d1_key], figures_dict[figure_d2_key] = figures_dict[figure_d2_key], figures_dict[figure_d1_key]
figures_dict = {key: f"{full_path}/{value}" for key, value in figures_dict.items()}

figures_dict

{'Figure 1-1. Hierarchy of PMI Project Risk Management Resources': '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images/page_14.png_roi_0_1.00.jpg',
 'Figure 1-2. Critical Success Factors for Project Risk Management': '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images/page_18.png_roi_0_0.96.jpg',
 'Figure 3-1. Project Risk Management Process Flow Diagram': '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images/page_29.png_roi_0_0.95.jpg',
 'Figure 4-1. Key Areas of Focus for the Plan Risk Management Process': '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images/page_35.png_roi_0_0.96.jpg',
 'Figure 5-1. Three Perspectives of Risk Identification': '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_cropped_images/page_39.png_roi_0_0.95.jpg',
 'Figure 5-2. Cause, Risk, and Effect': '/content/drive/MyDrive/PRM/PRM-Conceptual-

### Extract references from the description

In [183]:
def extract_references(df, figures_dict):
    df['References'] = None
    references_for_row_82 = None
    for index, row in df.iterrows():
        ref_set = set()
        matches = re.finditer(r'\s(?:Figure [1-9]-[1-9]|Figure [A-Z][1-9]+|Table D[1-9])', row['Description'])
        for match in matches:
            match_key = match.group(0).strip()
            match_words = match_key.split()[:2]
            matching_keys = [key for key in figures_dict.keys() if key.startswith(' '.join(match_words))]
            if matching_keys:
                if 'Table D' in match_key:
                    table_match_keys = [key for key in figures_dict.keys() if key.startswith(match_key)]
                    ref_set.update(figures_dict[key] for key in table_match_keys)
                else:
                    ref_set.add(figures_dict[matching_keys[0]])
        if index == 82:
            references_for_row_82 = list(ref_set)
            df.at[index, 'References'] = []
        elif index == 130:
            third_reference = None
            for i, path in enumerate(ref_set):
                if i == 2:
                    third_reference = path
            df.at[index, 'References'] = [third_reference] if third_reference is not None else None
        elif index == 151:
            first_reference = next(iter(ref_set), None)
            df.at[index, 'References'] = [first_reference] if first_reference is not None else None
        elif index == 152 or index == 148:
            last_reference = None
            for path in ref_set:
                last_reference = path
            df.at[index, 'References'] = [last_reference] if last_reference is not None else None
        elif index == 158:
            first_reference = next(iter(ref_set), None)
            df.at[index, 'References'] = list(ref_set - {first_reference})
            df.at[index - 1, 'References'] = [first_reference] if first_reference is not None else None
        else:
            df.at[index, 'References'] = list(ref_set)
    df.at[92, 'References'] = references_for_row_82
    return df

In [184]:
data=extract_references(data,figures_dict)
data.head()

,Concept,Description,Cleaned_Description,References
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[]
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,[/content/drive/MyDrive/PRM/PRM-Conceptual-Gra...
2,Project Risk Management Definition,"The definition of Project Risk Management, as ...",definition project risk management defined pmb...,[]
3,Role of Project Risk Management in Project Ma...,Project Risk Management is not an optional act...,project risk management optional activity esse...,[]
4,Good Risk Management Practice,Project Risk Management is a valuable componen...,project risk management valuable component pro...,[]


### Change References to URL's Using ImgBB

In [185]:
with open('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/pmi_img_bb.txt', 'r') as file:
    urls_bb_pmi = file.read().splitlines()

In [186]:
image_name_to_url = {}
for line in urls_bb_pmi:
    url = line.strip()
    # Extract the image name (e.g., 'page-18-png-roi-0-0-96.jpg') from the URL
    image_name = url.split('/')[-1][5:-15]
    image_name_to_url[image_name] = url

In [187]:
def replace_references(references):
    new_references = []
    for reference in references:
        image_name = reference.split('/')[-1]
        imgbb_url = image_name_to_url.get(image_name[5:-15].replace('.','-'))
        if imgbb_url:
            new_references.append(imgbb_url)
        else:
            new_references.append(reference)
    return new_references

In [188]:
data['References'] = data['References'].apply(replace_references)
data['References'].head()

0                                                   []
1    [https://i.ibb.co/4JjwzCR/page-14-png-roi-0-1-...
2                                                   []
3                                                   []
4                                                   []
Name: References, dtype: object

In [189]:
data.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_Clean.csv', index=False)

# MERGING PMBOK + PMI

In [190]:
pmbok_d = pd.read_csv("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMBOK_Clean.csv")
pmbok_d.shape[0]

29

In [191]:
pmi_d = pd.read_csv("/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_Clean.csv")
pmi_d.shape[0]

176

In [192]:
dfs = [pmbok_d, pmi_d]
pmi_pmbok = pd.concat(dfs)
pmi_pmbok.sample(5)

,Concept,Description,Cleaned_Description,References
67,Effective Presentation of Quantitative Analys...,Results from the quantitative tools are genera...,results quantitative tools generally available...,['https://i.ibb.co/kXmh40D/page-53-png-roi-0-0...
89,Response Identification,Risk response planning builds on the available...,risk response planning builds available inform...,[]
33,Complete Risk Statement,Identified risks should be clearly and unambig...,identified risks clearly unambiguously describ...,[]
108,Documenting the Results of the Monitor and Co...,The final control action of risk monitoring an...,final control action risk monitoring controlli...,[]
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[]


## NLP(Natural Language Processing)

### Sentence Tokenization and Removing Redanduncy
* We used sentence tokenization to split text into sentences because it helps in analyzing and processing text at a finer level, making it easier to extract meaning from individual statements.
* Lemmatization is used to reduce words to their base or dictionary form (lemma), which standardizes variations of a word and ensures that different forms of a word are treated as the same token. This is crucial for improving accuracy in text analysis and information retrieval. (this helped to remove redundancy because we have encountered issues like => "risks" != "risk")

In [193]:
def remove_repeated_and_plural_words(list_of_sentences):
    unique_sentences = []
    lemmatizer = WordNetLemmatizer()
    for sentence in list_of_sentences:
        words = sentence.split()
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        unique_sentence = ' '.join(lemmatized_words)
        unique_sentences.append(unique_sentence)
    return unique_sentences

In [194]:
#Sentence Tokenization using Regex because we have sections that are numbers concatenated by dots e.g: 11.2.3
pmi_pmbok['Sentences'] = pmi_pmbok['Cleaned_Description'].astype(str).replace('”','').apply(lambda x: re.split(r'(?<!\d)\.(?!\d)', x))
pmi_pmbok['Sentences'] = pmi_pmbok['Sentences'].apply(remove_repeated_and_plural_words)
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,References,Sentences
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,['https://i.ibb.co/rssfq3x/401.jpg'],[project risk management includes process cond...
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,['https://i.ibb.co/fXjdfx5/402.jpg'],[plan risk management process defining conduct...
2,Plan Risk Management Inputs,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[],[11.1.1.1 project charter described section 4....
3,Plan Risk Management Tools and Techniques,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...",[11.1.2.1 expert judgment described section 4....
4,Plan Risk Management Outputs,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/47Jwqzc/409.jpg', 'https://...",[11.1.3.1 risk management planthe risk managem...


### POS TAGGING
POS (Part-of-Speech) tagging is used to label words in a text with their grammatical category (e.g., noun, verb, adjective) to understand the syntactic structure and relationships within sentences.
In our case we need the NOUNs to use as Concepts and VERBS as relationships (so we can create graphs)



In [195]:
def pos_tagging(list_of_sentences):
    pos_tagged_sentences = []
    nouns_and_verbs_sentences = []
    for sentence in list_of_sentences:
        try:
            words = nltk.word_tokenize(sentence)
            pos_tags = nltk.pos_tag(words)
            # Filter to keep only nouns (NN, NNS, NNP, NNPS) and verbs (VB, VBD, VBG, VBN, VBP, VBZ)
            filtered_pos_tags = [(word, tag) for word, tag in pos_tags if tag.startswith('N') or tag.startswith('V')]
            if filtered_pos_tags:
                nouns_and_verbs_sentences.append(sentence)
            pos_tagged_sentences.append(filtered_pos_tags)
        except Exception as e:
            print(f"Error processing sentence: {sentence}")
            continue
    return pos_tagged_sentences, nouns_and_verbs_sentences

In [196]:
pmi_pmbok['POS_Tags[Nouns_Verbs]'] = pmi_pmbok['Sentences'].apply(pos_tagging)
pmi_pmbok['POS_Tags'] = pmi_pmbok['POS_Tags[Nouns_Verbs]'].apply(lambda x: x[0])
pmi_pmbok['Sentences[Nouns_Verbs]'] = pmi_pmbok['POS_Tags[Nouns_Verbs]'].apply(lambda x: x[1])
pmi_pmbok.drop('POS_Tags[Nouns_Verbs]', axis=1, inplace=True)
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,References,Sentences,POS_Tags,Sentences[Nouns_Verbs]
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,['https://i.ibb.co/rssfq3x/401.jpg'],[project risk management includes process cond...,"[[(project, NN), (risk, NN), (management, NN),...",[project risk management includes process cond...
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,['https://i.ibb.co/fXjdfx5/402.jpg'],[plan risk management process defining conduct...,"[[(plan, NN), (risk, VBP), (management, NN), (...",[plan risk management process defining conduct...
2,Plan Risk Management Inputs,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[],[11.1.1.1 project charter described section 4....,"[[(project, NN), (charter, NN), (described, VB...",[11.1.1.1 project charter described section 4....
3,Plan Risk Management Tools and Techniques,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...",[11.1.2.1 expert judgment described section 4....,"[[(judgment, NN), (described, VBD), (section, ...",[11.1.2.1 expert judgment described section 4....
4,Plan Risk Management Outputs,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/47Jwqzc/409.jpg', 'https://...",[11.1.3.1 risk management planthe risk managem...,"[[(risk, NN), (management, NN), (planthe, NN),...",[11.1.3.1 risk management planthe risk managem...


In [197]:
pmi_pmbok.drop('Sentences', axis = 1, inplace =True)
pmi_pmbok.rename(columns={'Sentences[Nouns_Verbs]':'Sentences'}, inplace=True)
pmi_pmbok.head(2)

,Concept,Description,Cleaned_Description,References,POS_Tags,Sentences
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,['https://i.ibb.co/rssfq3x/401.jpg'],"[[(project, NN), (risk, NN), (management, NN),...",[project risk management includes process cond...
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,['https://i.ibb.co/fXjdfx5/402.jpg'],"[[(plan, NN), (risk, VBP), (management, NN), (...",[plan risk management process defining conduct...


### Extracting KeyWords Using KeyBert
Extracting keywords using KeyBert helps identify important concepts and themes in text, which is essential for creating a meaningful conceptual graph. KeyBert leverages contextual embeddings to find contextually relevant keywords, ensuring that the graph is based on the most significant terms. This process enhances the quality of the recommendation system, as it relies on a well-structured and semantically rich graph to make informed suggestions.

In [198]:
kw_model = KeyBERT(model='all-mpnet-base-v2')
def extract_concepts_keybert(text):
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1,4),
        stop_words='english',
        highlight=False,
        top_n=10,
    )
    unique_keywords = set(keyword[0] for keyword in keywords)
    unique_keywords_list = list(unique_keywords)
    return unique_keywords_list

In [199]:
pmi_pmbok['KeyWords_KeyBert'] =  pmi_pmbok['Cleaned_Description'].astype(str).apply(extract_concepts_keybert)
pmi_pmbok[['Concept', 'KeyWords_KeyBert']].sample(5)

,Concept,KeyWords_KeyBert
6,Conclusion,"[project risk management described, project ri..."
62,Tools and Techniques for the Perform Quantita...,"[quantitative risk analysis characteristics, u..."
88,Tools and Techniques for the Plan Risk Respon...,"[involved planning risk responses, strategy im..."
82,Risk Response Strategies,"[project level risks overview, risks project, ..."
150,Techniques for Perform Qualtitative Risk Anal...,"[project risks analysis, project risks, priori..."


In [200]:
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,References,POS_Tags,Sentences,KeyWords_KeyBert
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,['https://i.ibb.co/rssfq3x/401.jpg'],"[[(project, NN), (risk, NN), (management, NN),...",[project risk management includes process cond...,"[project risk management, project risk managem..."
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,['https://i.ibb.co/fXjdfx5/402.jpg'],"[[(plan, NN), (risk, VBP), (management, NN), (...",[plan risk management process defining conduct...,"[project risk management, conduct risk managem..."
2,Plan Risk Management Inputs,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[],"[[(project, NN), (charter, NN), (described, VB...",[11.1.1.1 project charter described section 4....,"[project risk management, responsibilities man..."
3,Plan Risk Management Tools and Techniques,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...","[[(judgment, NN), (described, VBD), (section, ...",[11.1.2.1 expert judgment described section 4....,"[risk management process project, meetings doc..."
4,Plan Risk Management Outputs,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/47Jwqzc/409.jpg', 'https://...","[[(risk, NN), (management, NN), (planthe, NN),...",[11.1.3.1 risk management planthe risk managem...,"[project risk management, required project ris..."


In [201]:
pmi_pmbok.drop('POS_Tags', axis=1, inplace=True)

### Extracting Synonyms Using SBert
Extracting synonyms is valuable for enhancing natural language processing tasks and information retrieval. It broadens the search scope, improves recall, and ensures that a variety of terms can be used to find the same content. Synonyms help capture more comprehensive and relevant information when searching or analyzing text data.

In [202]:
def Synonym_extraction_SBert(df,title_column,keywords_column):
  All_Synonyms = []
  All_KeyWords = []
  for index, row in df.iterrows():
      sentences1 = [row[title_column]]
      sentences2 = row[keywords_column]
      embeddings1 = Sbert.encode(sentences1, convert_to_tensor=True)
      test = []
      synonym = []
      keywords = []
      for i in sentences2:
          embeddings2 = Sbert.encode(i, convert_to_tensor=True)
          cosine_scores = util.cos_sim(embeddings1, embeddings2)
          score = cosine_scores[0][0].item()
          if score >= 0.8:
              synonym.append(i)
          else:
              keywords.append(i)
          test.append(f'{sentences1[0]} <=> {i} Has the score: {score}')
      All_Synonyms.append(synonym)
      All_KeyWords.append(keywords)
  return All_Synonyms, All_KeyWords

In [203]:
pmi_pmbok['Concept'].fillna('', inplace=True)
pmi_pmbok['KeyWords_KeyBert'].fillna('', inplace=True)

In [205]:
pmi_pmbok['Synonyms_SBert'],pmi_pmbok['KeyWords_SBert'] = Synonym_extraction_SBert(pmi_pmbok,'Concept','KeyWords_KeyBert')
pmi_pmbok.head()

,Concept,Description,Cleaned_Description,References,Sentences,KeyWords_KeyBert,Synonyms_SBert,KeyWords_SBert
0,PROJECT RISK MANAGEMENT,Project Risk Management includes the processes...,project risk management includes processes con...,['https://i.ibb.co/rssfq3x/401.jpg'],[project risk management includes process cond...,"[project risk management, project risk managem...","[project risk management, project risk managem...",[]
1,Plan Risk Management,Plan Risk Management is the process of definin...,plan risk management process defining conduct ...,['https://i.ibb.co/fXjdfx5/402.jpg'],[plan risk management process defining conduct...,"[project risk management, conduct risk managem...","[plan risk management, plan risk management pr...","[project risk management, conduct risk managem..."
2,Plan Risk Management Inputs,11.1.1.1 PROJECT CHARTER Described in Section ...,11.1.1.1 project charter described section 4.1...,[],[11.1.1.1 project charter described section 4....,"[project risk management, responsibilities man...",[],"[project risk management, responsibilities man..."
3,Plan Risk Management Tools and Techniques,11.1.2.1 EXPERT JUDGMENT Described in Section...,11.1.2.1 expert judgment described section 4.1...,"['https://i.ibb.co/RH2d7J4/406.png', 'https://...",[11.1.2.1 expert judgment described section 4....,"[risk management process project, meetings doc...",[risk management plan developed],"[risk management process project, meetings doc..."
4,Plan Risk Management Outputs,11.1.3.1 RISK MANAGEMENT PLANThe risk manageme...,11.1.3.1 risk management planthe risk manageme...,"['https://i.ibb.co/47Jwqzc/409.jpg', 'https://...",[11.1.3.1 risk management planthe risk managem...,"[project risk management, required project ris...",[],"[project risk management, required project ris..."


In [211]:
pmi_pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK.csv', index=False)

### Relation Extraction Using Rebel

In [ ]:
pmi_pmbok = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK.csv')

### Definition Extraction

In [ ]:
# ---------------------> Need Token Key
# API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-xxl"
# headers = {"Authorization": "API TOKEN"}

# def query(payload):
# 	response = requests.post(API_URL, headers=headers, json=payload)
# 	return response.json()
# definition = []
# for concept , desc in zip(df['Concept'],df["Description"]):
#   output = query({
#       "inputs" : f"given a concept and its description give me the definition of the concept , concept :{concept} , descripton :{desc} "
#   })
#   if 'error' in output:
#     output = desc
#   definition.append(output)
#   print(output)

In [ ]:
# l = []
# for desc in pmi_pmbok["Definition"]:
#     try:
#         for j in ast.literal_eval(desc):
#             l.append(j['generated_text'])
#     except Exception as e:
#         l.append(desc)

In [ ]:
# Here's the Resulting DF
pmi_pmbok_f = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK_difinition.csv')
pmi_pmbok_f.head()

In [ ]:
test = pmi_pmbok_f
test.head(2)

,Concept,Description,Cleaned_Description,References,Sentences,POS_Tags,KeyWords_KeyBert,Synonyms_SBert,KeyWords_SBert,Definition
0,INTRODUCTION,Project Management Institute PMI practice stan...,project management institute pmi practice stan...,[],['project management institute pmi practice st...,"[[('project', 'NN'), ('management', 'NN'), ('i...","['pmi practice standards', 'pmi practice stand...",[],"['pmi practice standards', 'pmi practice stand...",A practice standard is a guide to the use of a...
1,Purpose of the Practice Standard for Project ...,The purpose of the Practice Standard for Proje...,purpose practice standard project risk managem...,['/content/drive/MyDrive/PRM/PRM-Conceptual-Gr...,['purpose practice standard project risk manag...,"[[('purpose', 'JJ'), ('practice', 'NN'), ('sta...","['project risk management', 'projects risk man...","['project risk management', 'projects risk man...","['introduction project risk', 'standard projec...",The purpose of the Practice Standard for Proje...


# Final Dataframe

In [ ]:
pmi_pmbok.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK_Final2.0.csv', index = False) # to run after relation extraction and lbe9i lkol

### Relation Etraction 2

In [212]:
test = pd.read_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/PMI_PMBOK.csv')

In [21]:
# !pip install python-dotenv
# !pip install datasets
# !pip install langchain
# !pip install transformers
# !pip install neo4j
# !pip install llama-index
# !pip install ipython-ngql nebula3-python networkx pyvis
# !pip install torch
# !pip install -U huggingface_hub

In [8]:
import os
import random
import json
import hashlib
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [9]:
validation_data, test_data = load_dataset("suolyer/pile_wikipedia", split=['validation', 'test'])

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [10]:
m = hashlib.md5()
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [17]:
test.columns

Index(['id', 'text'], dtype='object')

In [213]:
t = []
def bert_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

def create_chunk_dataset(content):
      m.update(content.encode('utf-8'))
      uid = m.hexdigest()[:12]
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size = 400,
          chunk_overlap  = 40,
          length_function = bert_len,
          separators=['\n\n', '\n', ' ', ''],
      )
      chunks = text_splitter.split_text(content)
      for i, chunk in enumerate(chunks):
          t.append({
              'id': f'{uid}-{i}',
              'text': chunk
          })

for dt in test['Sentences']:
    create_chunk_dataset(dt)

In [214]:
filename = '/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/test1.jsonl'
with open(filename, 'w') as outfile:
    for x in t:
        outfile.write(json.dumps(x) + '\n')

In [216]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

In [25]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 1,
}

triples = []

In [217]:
def generate_triples(texts):

  model_inputs = tokenizer(texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
  generated_tokens = model.generate(
      model_inputs["input_ids"].to(model.device),
      attention_mask=model_inputs["attention_mask"].to(model.device),
      **gen_kwargs
  )
  decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
  for idx, sentence in enumerate(decoded_preds):
      et = extract_triplets(sentence)
      for t in et:
        triples.append((t['head'], t['type'], t['tail']))

for i in tqdm(range(0, len(t), 2)):
  try:
    texts = [t[i]['text'], t[i+1]['text']]
  except:
    texts = [t[i]['text']]
  generate_triples(texts)

  0%|          | 0/245 [00:00<?, ?it/s]

In [218]:
distinct_triples = list(set(triples))

In [226]:
#/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/test1.jsonl
with open('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/rebel_triples.json', 'w') as file:
    json.dump(distinct_triples, file)

# load
with open('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/rebel_triples.json', 'r') as file:
    loaded_triples = json.load(file)
    print(loaded_triples)

[['risk management plan', 'use', 'risk management'], ['risk category', 'subclass of', 'risk breakdown structure'], ['resource calendar', 'facet of', 'resource'], ['risk response converse effect', 'opposite of', 'risk response appropriate significance risk'], ['risk', 'studied by', 'risk analysis'], ['expert judgment', 'subclass of', 'knowledge'], ['risk uncertain event', 'subclass of', 'risk'], ['risk interdependency', 'subclass of', 'feedback loop'], ['open inclusive', 'instance of', 'cultural issue'], ['identify risks', 'facet of', 'risk'], ['cause effect risk', 'subclass of', 'risk'], ['critical chain', 'subclass of', 'project management'], ['monte carlo analysis', 'use', 'simulation'], ['root cause risk', 'subclass of', 'risk'], ['project file', 'has part', 'data'], ['risk driver', 'facet of', 'risk'], ['uncertainty', 'has effect', 'cause effect'], ['monitoring', 'subclass of', 'planning'], ['cost', 'has effect', 'revenue'], ['quantitative risk analysis', 'part of', 'risk analysis'

In [227]:
len(loaded_triples)

616

In [220]:
loaded_triples[:5]

[['risk management plan', 'use', 'risk management'],
 ['risk category', 'subclass of', 'risk breakdown structure'],
 ['resource calendar', 'facet of', 'resource'],
 ['risk response converse effect',
  'opposite of',
  'risk response appropriate significance risk'],
 ['risk', 'studied by', 'risk analysis']]

In [228]:
relations = pd.DataFrame(loaded_triples, columns=["Subject", "Relation", "Object"])
relations.head(10)

,Subject,Relation,Object
0,risk management plan,use,risk management
1,risk category,subclass of,risk breakdown structure
2,resource calendar,facet of,resource
3,risk response converse effect,opposite of,risk response appropriate significance risk
4,risk,studied by,risk analysis
5,expert judgment,subclass of,knowledge
6,risk uncertain event,subclass of,risk
7,risk interdependency,subclass of,feedback loop
8,open inclusive,instance of,cultural issue
9,identify risks,facet of,risk


In [234]:
relations.sample(15)

,Subject,Relation,Object
120,contingency planning,has part,risk response
555,non-linear utility function,opposite of,linear utility function
426,decision tree,has part,chance nodes
23,risk,facet of,risk
323,root cause,subclass of,cause effect
70,industry knowledge base,subclass of,knowledge
407,quantitative risk analysis,subclass of,analysis
295,cause effect,part of,ishikawa diagram
480,project management institute,field of work,project management
531,risk,has effect,threat


In [229]:
relations.shape[0]

616

In [235]:
relations.to_csv('/content/drive/MyDrive/PRM/PRM-Conceptual-Graphs-Recommendations/src/relations.csv', index=False)